ติดตั้ง pip สำหรับติดตั้ง Python Package

In [1]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [2]:
!pip install streamlit_folium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.0/326.0 kB 4.7 MB/s eta 0:00:00


In [3]:
! pip -q install pythainlp python-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.8 MB/s eta 0:00:00


เขียนเว็บแอป



In [32]:

%%writefile my_streamlit_app.py
import streamlit as st
from tensorflow import keras
import pandas as pd
import re
from pythainlp.util import normalize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import word_tokenize
from pythainlp import word_vector
import numpy as np
from functools import reduce
wv = word_vector.WordVector()
word2vec = wv.get_model()


# Create a flag to check if the model has been loaded successfully

model_loaded = False
try:
    # Load the model
    model = keras.models.load_model('my_model.h5')
    model_loaded = True
except:
    st.warning('Model loading failed. Please check the model file.')


def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


def clean_me(data):
  stopwords = list(thai_stopwords())
  stopwords.append("nan")
  stopwords.append("-")
  stopwords.append("_")
  stopwords.append("")
  stopwords.append(" ")

  data['clean_text'] = data['text'].str.replace(r'<[^<>]*>', '', regex=True)
  data['clean2_text']= data['clean_text'].str.strip().str.lower().str.replace('\r+', ' ').str.replace('\n+',' ').str.replace('\t+',' ')
  data['clean3_text'] = data.apply(lambda row: deEmojify(row['clean2_text']), axis=1)
  # Normalize text
  data['clean4_text'] = data.apply(lambda row: normalize(row['clean3_text']), axis=1)
  # Word segmentation: it will take a while....
  data['wordseged_text'] = data.apply(lambda row: word_tokenize(row['clean4_text'], engine="newmm-safe"), axis=1)
  # Join the wordsegged with space
  data['wordseged_space_text'] = data.apply(lambda row: " ".join(row["wordseged_text"]), axis=1)


  return(data)

def sentences_to_indices(X, word2vec, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()`.
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word2vec -- a trained Word2Vec model from gensim
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this.
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """

    m = X.shape[0]                                   # number of training examples

    # Initialize X_indices as a numpy matrix of zeros and the correct shape
    X_indices = np.zeros((m, max_len))

    for i in range(m):                               # loop over training examples

        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        # print(X)
        # print(len(X[i].lower().split()))
        sentence_words = X[i].lower().split()[:max_len]

        # Initialize j to 0
        j = 0
        try:
        # Loop over the words of sentence_words
          for w in sentence_words:
              # Set the (i,j)th entry of X_indices to the index of the correct word.

                if w in word2vec.key_to_index:
                    X_indices[i, j] = word2vec.key_to_index[w]
                    # Increment j to j + 1
                    j += 1
        except:
              print('key error: ', w)


    return X_indices



def combine(a, b):
    data = pd.DataFrame()
    lines = str.split(a, '\n')
    if b != "":
        lines = get_comments(b)
        if lines == []:
            text001 = 'CANNOT_GET DATA from Youtube'
            print(text001)

    data['text'] = lines
    data = clean_me(data)
    a = data['wordseged_space_text'][0] + ' SENTIMENT: '

    X_train_indices = sentences_to_indices(data['wordseged_space_text'].values, word2vec, 128)
    result = model.predict(X_train_indices[:])
    txt_pos, txt_neu, txt_neg, fig = pretty_output(lines,result)

    return txt_pos, txt_neu, txt_neg, fig


def mirror(x):
    return x




# Streamlit app
# เรียกใช้งาน Bootstrap CSS โดยใส่ URL ของ CSS ใน markdown
st.markdown("""
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3" crossorigin="anonymous">
""", unsafe_allow_html=True)


import streamlit as st

# เริ่มต้นโค้ด CSS
st.markdown("""
    <style>
        /* กำหนดให้หัวข้อตรงกลาง */
        .title {
            color: #31906E; /* กำหนดสีของข้อความ */
            text-align: center;
        }
    </style>
""", unsafe_allow_html=True)

# แสดงหัวข้อ
st.markdown("<h1 class='title'>Thai Sentiment Analysis App</h1>", unsafe_allow_html=True)




# ใช้ CSS เพื่อกำหนดสีและจัดให้ข้อความอยู่ตรงกลาง
st.markdown("""
    <style>
        /* กำหนดสีและจัดให้ข้อความอยู่ตรงกลาง */
        .centered-text {
            color: #A8A9AD; /* กำหนดสีของข้อความ */
            text-align: center; /* จัดให้ข้อความอยู่ตรงกลาง */
            font-size: 19px; /* กำหนดขนาดตัวอักษร */
        }
    </style>
""", unsafe_allow_html=True)

# แสดงข้อความ
st.markdown("<p class='centered-text'>Created by</p>", unsafe_allow_html=True)


# แสดงข้อความ
st.markdown("<p class='centered-text'>Kamolwan Artsungnoen and Yonlada Mattra</p>", unsafe_allow_html=True)
st.markdown("<p class='centered-text'>Department of Computer Science, College of Computing, Khon Kaen University</p>", unsafe_allow_html=True)



# แสดงรูปภาพจาก URL
image_url = "https://i.pinimg.com/564x/32/8e/00/328e00ebb073d75ed306d14af670fda2.jpg"
st.image(image_url, caption='ใช้เพื่อการศึกษา : https://i.pinimg.com/564x/32/8e/00/328e00ebb073d75ed306d14af670fda2.jpg', use_column_width=True)

# Input text
text_input = st.text_area("Enter Thai text for sentiment analysis", "กรุณากรอกข้อความ... ")

st.markdown("""
<style>
    /* กำหนดสีของปุ่มเมื่อปุ่มถูกกด */
    .stButton>button {
        background-color: #AAF0D1; /* กำหนดสี */
        color: black; /* กำหนดสีของตัวอักษร */
        font-weight: bold; /* กำหนดตัวหนา */
    }
    /* กำหนดสีของปุ่มเมื่อเมาส์ไปวางบนปุ่ม */
    .stButton>button:hover {
        background-color: #3EB489; /* กำหนดสี */
                color: black; /* กำหนดสีของตัวอักษร */
        font-weight: bold; /* กำหนดตัวหนา */
    }
</style>
""", unsafe_allow_html=True)

# Button to trigger analysis
if st.button("Analyze Sentiment"):
    data = pd.DataFrame()
    data['text'] = [text_input]
    data = clean_me(data)
    X_train_indices = sentences_to_indices(data['wordseged_space_text'].values, word2vec, 128)
    results = model.predict(X_train_indices[:]) #พรีดิก
    # Analyze sentiment using the model
    #results = sentiment_analyzer(text_input)
    #print (results)



# Extract sentiment and score

    label = np.array(['negative😡', 'neutral😐', 'positive🙂'])
    txt_sentiment = label[np.argmax(results, axis=1)]
    sentiment = txt_sentiment[0]
    score = results[0]

    st.write(f'{score} {sentiment}') #เพิ่มตัวอิโมจิหลังข้อความ

    # Display result as progress bars
    #st.subheader("Sentiment Analysis Result:")




Overwriting my_streamlit_app.py


In [5]:
!wget -q -O - ipv4.icanhazip.com

34.86.58.234


In [33]:
! streamlit run my_streamlit_app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.58.234:8501

npx: installed 22 in 2.831s
your url is: https://ten-cooks-kick.loca.lt
  Stopping...
^C
